# Dense Retriver 실험

In [9]:
import torch
from torch.utils.data import DataLoader

from dense_retrieval import DenseRetrieval

In [10]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from typing import List, Tuple, NoReturn, Any, Optional, Union

import torch
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from datasets import Dataset, load_from_disk, concatenate_datasets

from retrieval import SparseRetrieval, timer

## 1. 데이터 로드

In [11]:
import os
import json 

data_path  = "../data/"
dataset_path = "../data/train_dataset"
context_path = "wikipedia_documents.json"
model_checkpoint = "klue/bert-base"

org_dataset = load_from_disk(dataset_path)
full_ds = concatenate_datasets([
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ])

with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

Loading cached processed dataset at ../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [19]:
df = pd.DataFrame(contexts, columns=['context']).to_csv('./data/contexts2.csv', index=True)
df.iloc[0]['context']

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [20]:
df_train = pd.DataFrame(org_dataset['train'])
#df_train = df_train[['document_id','title','answers','question','context', 'id','__index_level_0__']]
df_train = df_train[['answers']]
df_train.to_csv('./data/train_answers.csv')

df_valid = pd.DataFrame(org_dataset['validation'])
#df_valid = df_valid[['document_id','title','answers','question','context', 'id','__index_level_0__']]
df_valid = df_valid[['answers']]
df_valid.to_csv('./data/valid_answers.csv')

In [27]:
len(contexts)

56737

[s for s in contexts if org_dataset["validation"][0]['context'][0:5] in s]

## 2. 추론

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=False,)

args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [30]:
dense_retriever = DenseRetrieval(tokenize_fn=tokenizer.tokenize, data_path = data_path, 
                                context_path = context_path, dataset_path=dataset_path, 
                                tokenizer=tokenizer, train_data=org_dataset["train"])
## 학습과정 ##
# train_dataset = dense_retriever.make_train_data(tokenizer)
# dense_retriever.init_model(model_checkpoint)
# dense_retriever.train(args, train_dataset)

## 추론과정 ##
dense_retriever.load_model(model_checkpoint, "outputs/p_encoder_3.pt", "outputs/q_encoder_3.pt")
dense_retriever.get_dense_embedding()
#df = dense_retriever.retrieve(full_ds[0]['question'], topk=3)

Lengths of unique contexts : 56737
Embedding pickle load.
cuda:0


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

load_model finished...


100%|██████████| 14184/14184 [14:29<00:00, 16.31it/s]


get_dense_embedding finished...


In [31]:
with open("./data/dense_embedding3.bin", "rb") as f:
    dense_retriever.dense_p_embedding = pickle.load(f)

import pickle

with open("./data/dense_embedding.bin", "wb") as f:
    pickle.dump(dense_retriever.dense_p_embedding, f)

In [32]:
dense_retriever.dense_p_embedding.shape

torch.Size([56736, 768])

## 3. 실험 결과 확인

In [33]:
dense_retriever.get_relevant_doc(org_dataset["train"]["question"][0], k=1)

(tensor([159.8050, 164.2354, 151.9531,  ..., 147.8784, 145.6791, 155.0440]),
 tensor([18999, 23598, 19002,  ..., 43548, 17540, 54222]))

In [34]:
result_retriever = dense_retriever.retrieve(org_dataset["validation"], topk=5)
result_retriever

100%|██████████| 60/60 [00:03<00:00, 16.47it/s]


[query exhaustive search dataset] done in 6.381 s


,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"[56287, 25599, 23610, 4593, 26147]","호르스트 제호퍼 주총리 밑에서 재무장관을 지냈으며, 주를 대변하는 연방 상원의원으로...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"[19569, 46742, 19566, 23426, 22775]",이 역이 교통 허브로 거듭나게 된 것은 거의 우연에 가까웠으며 기존에 스카버러 타운...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"[50560, 11505, 32700, 48089, 11305]",이번 참의원의원 선거에서는 지역구인 이와테현 선거구에서 1명(야권 단일 무소속 후보...,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"[34735, 47277, 13936, 35175, 33567]",머리 전체와 목부위까지 감싸는 형태인 아멧은 일부를 풀었다 잠그는 형태로 입고 벗었...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"[29185, 48493, 24059, 9353, 47823]",탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"[51337, 6959, 25002, 32096, 32097]","어느 날의 대낮, 적국이었던 마레와 강화협정을 맺은 것으로 보이는 엘디아에 마레의 ...","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"[17689, 20396, 21892, 4615, 19173]","인필드 플라이 규칙(Infield Fly rule)는 무사, 1사 1,2루 혹은 만...",현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"[9802, 9962, 6592, 15386, 15507]","1356년, 우웨이스 1세는 아버지인 하산 부즈루그의 뒤를 이어 즉위했다. 1357...",1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"[6959, 41918, 52504, 24294, 42143]",페르시아가 그리스 역사에 등장한 것은 페르시아가 뤼디아와 이오니아의 몇몇 그리스계 ...,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"


In [35]:
for i in range(10):
    df = dense_retriever.retrieve(org_dataset['validation'][i]['question'], topk=3)
    print(df)

[Search query]
 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? 

Top-1 passage with score 125.999008
호르스트 제호퍼 주총리 밑에서 재무장관을 지냈으며, 주를 대변하는 연방 상원의원으로서 상원 재무위원회 소속이었다.\n\n재무장관 시절 유럽 연합 집행위원회의 일괄 지원을 받고자 부실 주 지원 대출은행인 바이에른LB의 재건을 감독하기도 했다. 2014년에는 바이에른LB를 압박하여 헝가리 측에 MKB 단위를 매각함으로서 20여년 간 20억 유로의 손실을 초래한 부실투자를 종식시키기도 했다. 2015년에는 한스 외르크 셸링 오스트리아 외무장관과 협상을 타결하여 하이포 알페아드리아뱅크 인터내셔널(케른텐주 지역 은행)의 붕괴에서 시작된 양측 정부의 법적 분쟁을 끝냈다. 양해 각서에 따르면 오스트리아는 바이에른주에 12억 3천만 유로를 지불하며, 모든 관련 소송은 취하되었다. \n\n2012년 죄더는 제호퍼 당시 주총리와 함께 연방헌법법원에 소송을 제기하여 바이에른처럼 부유한 주가 전국의 부실경제 구제 차원에서 재정이전을 하도록 하는 독일 시스템 점검을 요구했다. 죄더의 제안에 따라 바이에른주 정부는 독일 최초로 폭스바겐을 상대로 배출가스 시험 사기 사건 관련 소송을 제기해 손해배상을 요구한 주가 되었다. 이 시기 죄더는 해당 스캔들로 인해 70만 유로에 달하는 공무원 연금기금 손실을 입었다고 밝혔다. \n\n2017년 총선 결과 기사련이 참패하면서 제호퍼는 대표직 사퇴 압력을 받게 되었고, 이에 그는 당대표직에서 물러나지는 않는 대신 바이에른주 총리직은 죄더에게 인계하겠다고 밝혔다.
Top-2 passage with score 125.162476
엑스포스는 1976 몬트리올 올림픽 메인 스타디움인 스타드 올랭피크를 28년간 사용했다. 그러나 원체 문제 투성이인 구장이 제대로 될 리가 만무. 애초에 올림픽 개막을 며칠 앞두고 겨우겨우 완공한 데다 여기저기 보수를 해야 할 정도로 상태가 영 좋지 않았다. 게다가 원래 종합경

In [36]:
topK_list = [1,10,20,50]
result = dense_retriever.topk_experiment(topK_list, org_dataset['validation'])
print(result)

100%|██████████| 60/60 [00:03<00:00, 16.04it/s]


[query exhaustive search dataset] done in 6.482 s


 25%|██▌       | 1/4 [00:06<00:19,  6.53s/it]

100%|██████████| 60/60 [00:03<00:00, 16.20it/s]


[query exhaustive search dataset] done in 6.453 s


...

 75%|███████▌  | 3/4 [00:19<00:06,  6.51s/it]

100%|██████████| 60/60 [00:03<00:00, 15.89it/s]


[query exhaustive search dataset] done in 6.521 s


100%|██████████| 4/4 [00:26<00:00,  6.52s/it]


{1: 0.12083333333333333, 10: 0.31666666666666665, 20: 0.4041666666666667, 50: 0.5375}
